<a href="https://colab.research.google.com/github/Hanbin-git/dacon_new_drug/blob/main/20250724_%EC%9D%BC%EB%B0%98%EC%A0%84%EB%9E%B5%20%EC%A0%81%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip -o "/content/drive/MyDrive/data.zip" -d "/content/data_1"

Archive:  /content/drive/MyDrive/data.zip
  inflating: /content/data_1/sample_submission.csv  
  inflating: /content/data_1/test.csv  
  inflating: /content/data_1/train.csv  


In [ ]:
# ==============================================================================
# ✅ 초기 환경 설정 및 라이브러리 설치 (가장 먼저 실행)
# ==============================================================================

# 1. pip 캐시를 모두 비웁니다.
print("Clearing pip cache...")
!pip cache purge

# 2. numpy와 scikit-learn 관련 패키지를 강제로 제거합니다.
print("Uninstalling numpy and scikit-learn related packages...")
!pip freeze | grep -E 'numpy|scikit-learn' | xargs -n 1 pip uninstall -y

# 3. Colab 기본 라이브러리 (TensorFlow, Numba 등)와 호환되는 특정 버전의 numpy와 scikit-learn을 설치합니다.
#    numpy를 1.26.x 대로 낮춰서 시도합니다.
print("Installing compatible versions of numpy and scikit-learn (attempting numpy 1.26.x)...")
!pip install numpy==1.26.4 scikit-learn==1.6.0 --no-cache-dir --quiet

# 4. 나머지 필요한 라이브러리들을 설치합니다.
print("Installing other necessary libraries...")
!pip install rdkit-pypi catboost xgboost lightgbm optuna --no-cache-dir --quiet

# 5. 파이썬 런타임을 강제로 재시작합니다. (가장 중요!)
print("Forcing runtime restart... Please wait a few seconds for reconnection.")
import os
os.kill(os.getpid(), 9)

# ==============================================================================

Clearing pip cache...
Files removed: 0
Uninstalling numpy and scikit-learn related packages...
Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
Installing compatible versions of numpy and scikit-learn (attempting numpy 1.26.x)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 358.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 26.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<

In [1]:
# numpy & scikit-learn 호환성 보장 버전으로 재설치
!pip install numpy==1.26.4 scikit-learn==1.5.0 --force-reinstall --quiet --no-cache-dir


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 118.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 237.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 357.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 132.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 364.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 287.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.0 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 whi

In [2]:
!pip install rdkit-pypi catboost xgboost lightgbm optuna --quiet --no-cache-dir


In [ ]:
import os
os.kill(os.getpid(), 9)


In [1]:
# ✅ 1. 라이브러리
import pandas as pd, numpy as np, optuna, time
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem, MACCSkeys, Lipinski
from catboost import CatBoostRegressor, Pool
import warnings
warnings.filterwarnings("ignore")

# ✅ 2. 데이터 경로 설정 (Colab)
train_path = "/content/data_1/train.csv"
test_path = "/content/data_1/test.csv"
submission_path = "/content/data_1/sample_submission.csv"

# ✅ 3. 데이터 로딩
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
submission = pd.read_csv(submission_path)

# ✅ 4. SMILES → Mol 변환
train["Mol"] = train["Canonical_Smiles"].apply(Chem.MolFromSmiles)
test["Mol"] = test["Canonical_Smiles"].apply(Chem.MolFromSmiles)

# ✅ 5. Feature 생성 함수 (RDKit + Fingerprint)
def get_features(df):
    features = []
    for mol in df["Mol"]:
        if mol is None:
            features.append(np.zeros(2048 + 10))
            continue
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
        descs = [
            Descriptors.MolWt(mol),
            Descriptors.MolLogP(mol),
            Descriptors.NumHAcceptors(mol),
            Descriptors.NumHDonors(mol),
            Descriptors.TPSA(mol),
            Descriptors.NumRotatableBonds(mol),
            Descriptors.RingCount(mol),
            Lipinski.NumAromaticRings(mol),
            Lipinski.NumAliphaticRings(mol),
            Lipinski.NumSaturatedRings(mol),
        ]
        features.append(np.concatenate([fp, descs]))
    return pd.DataFrame(features)

X = get_features(train)
X_test = get_features(test)
y = train["Inhibition"]

# ✅ 6. Optuna 튜닝 Objective
def objective(trial):
    params = {
        "depth": trial.suggest_int("depth", 4, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "random_strength": trial.suggest_float("random_strength", 1e-9, 10.0),
        "iterations": 3000,
        "task_type": "GPU",
        "devices": "0",
        "verbose": 0,
        "early_stopping_rounds": 50
    }

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    maes = []
    for train_idx, valid_idx in kf.split(X):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        model = CatBoostRegressor(**params)
        model.fit(X_train, y_train, eval_set=(X_valid, y_valid), use_best_model=True)
        preds = model.predict(X_valid)
        mae = mean_absolute_error(y_valid, preds)
        maes.append(mae)

    return np.mean(maes)

# ✅ 7. Optuna 튜닝 수행
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)
print("✅ Best Params:", study.best_params)

# ✅ 8. 최적 파라미터로 전체 학습
best_params = study.best_params
best_params.update({
    "iterations": 3000,
    "task_type": "GPU",
    "devices": "0",
    "early_stopping_rounds": 50,
    "verbose": 500
})

final_model = CatBoostRegressor(**best_params)
final_model.fit(X, y)

# ✅ 9. 예측 및 제출
preds = final_model.predict(X_test)
submission["Inhibition"] = preds
submission.to_csv("submission_catboost_optuna.csv", index=False)
print("✅ 제출 파일 저장 완료: submission_catboost_optuna.csv")


[I 2025-07-24 00:27:59,607] A new study created in memory with name: no-name-22f289ce-d8e0-4afe-aead-b4500a19d4a1
[I 2025-07-24 00:28:23,738] Trial 0 finished with value: 19.848299539685012 and parameters: {'depth': 10, 'learning_rate': 0.16941918122839386, 'l2_leaf_reg': 6.679162970668696, 'bagging_temperature': 0.03946037037149386, 'random_strength': 7.516664402127794}. Best is trial 0 with value: 19.848299539685012.
[I 2025-07-24 00:28:36,897] Trial 1 finished with value: 19.78045688863532 and parameters: {'depth': 5, 'learning_rate': 0.0666383385660528, 'l2_leaf_reg': 6.531849169898688, 'bagging_temperature': 0.33896922882559577, 'random_strength': 3.3643060129438322}. Best is trial 1 with value: 19.78045688863532.
[I 2025-07-24 00:28:49,823] Trial 2 finished with value: 19.858360940764676 and parameters: {'depth': 5, 'learning_rate': 0.05870742230548659, 'l2_leaf_reg': 2.4663907758442742, 'bagging_temperature': 0.5741414513157761, 'random_strength': 4.095244228038913}. Best is tri

✅ Best Params: {'depth': 5, 'learning_rate': 0.10241282624747099, 'l2_leaf_reg': 6.490954192088343, 'bagging_temperature': 0.8175774635077507, 'random_strength': 3.844677040758154}
0:	learn: 26.0862837	total: 6.28ms	remaining: 18.8s
500:	learn: 16.6861759	total: 2.7s	remaining: 13.5s
1000:	learn: 13.9339702	total: 5.44s	remaining: 10.9s
1500:	learn: 12.0522049	total: 8.12s	remaining: 8.11s
2000:	learn: 10.5858835	total: 10.8s	remaining: 5.4s
2500:	learn: 9.4968768	total: 13.5s	remaining: 2.69s
2999:	learn: 8.7179855	total: 16.2s	remaining: 0us
✅ 제출 파일 저장 완료: submission_catboost_optuna.csv


In [1]:
# ✅ 라이브러리 import
import pandas as pd, numpy as np, shap, os
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor, Pool

# ✅ 데이터 경로
DATA_PATH = "/content/data_1/"
train = pd.read_csv(DATA_PATH + "train.csv")
test = pd.read_csv(DATA_PATH + "test.csv")
submission = pd.read_csv(DATA_PATH + "sample_submission.csv")

# ✅ 전처리: SMILES → RDKit Mol
from rdkit import Chem
train["Mol"] = train["Canonical_Smiles"].apply(Chem.MolFromSmiles)
test["Mol"] = test["Canonical_Smiles"].apply(Chem.MolFromSmiles)

# ✅ 기본 분자 특징 + 파생 변수 생성 함수
from rdkit.Chem import Descriptors, rdMolDescriptors, Crippen

def get_features(df):
    features = {
        "MolWt": [],
        "TPSA": [],
        "NumRotatableBonds": [],
        "NumHAcceptors": [],
        "NumHDonors": [],
        "LogP": [],
    }

    for mol in df["Mol"]:
        features["MolWt"].append(Descriptors.MolWt(mol))
        features["TPSA"].append(rdMolDescriptors.CalcTPSA(mol))
        features["NumRotatableBonds"].append(Descriptors.NumRotatableBonds(mol))
        features["NumHAcceptors"].append(rdMolDescriptors.CalcNumHBA(mol))
        features["NumHDonors"].append(rdMolDescriptors.CalcNumHBD(mol))
        features["LogP"].append(Crippen.MolLogP(mol))

    features_df = pd.DataFrame(features)

    # ✅ 파생 변수 추가
    features_df["MolWt_TPSA"] = features_df["MolWt"] * features_df["TPSA"]

    return features_df

# ✅ 특성 추출
X_train_full = get_features(train)
X_test = get_features(test)
y = train["Inhibition"]

# ✅ SHAP을 이용한 상위 변수 선택
model = CatBoostRegressor(verbose=0, random_state=42)
model.fit(X_train_full, y)
explainer = shap.Explainer(model)
shap_values = explainer(X_train_full)

# 상위 5개 변수 선택
shap_sum = np.abs(shap_values.values).mean(axis=0)
top_features = X_train_full.columns[np.argsort(shap_sum)[::-1][:50]]

X_train = X_train_full[top_features]
X_test = X_test[top_features]

# ✅ 최적 CatBoost 파라미터
best_params = {
    "depth": 5,
    "learning_rate": 0.1024,
    "l2_leaf_reg": 6.49,
    "bagging_temperature": 0.82,
    "random_strength": 3.84,
    "iterations": 3000,
    "eval_metric": "MAE",
    "loss_function": "MAE",
    "verbose": 0,
    "task_type": "GPU"
}

# ✅ Cross-Validation + Soft Voting 앙상블
kf = KFold(n_splits=5, shuffle=True, random_state=42)
test_preds = np.zeros(len(X_test))
val_scores = []

for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train)):
    X_tr, X_val = X_train.iloc[tr_idx], X_train.iloc[val_idx]
    y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

    model = CatBoostRegressor(**best_params)
    model.fit(X_tr, y_tr, eval_set=(X_val, y_val), early_stopping_rounds=100, verbose=500)

    val_pred = model.predict(X_val)
    mae = mean_absolute_error(y_val, val_pred)
    val_scores.append(mae)
    print(f"✅ Fold {fold+1} MAE: {mae:.4f}")

    test_preds += model.predict(X_test) / kf.n_splits

print(f"\n✅ 평균 MAE: {np.mean(val_scores):.4f}")

# ✅ 제출 파일 생성
submission["Inhibition"] = test_preds
submission.to_csv("submission_catboost_optuna_shap.csv", index=False)
print("📁 제출 파일 저장 완료: submission_catboost_optuna_shap.csv")


Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 21.9165548	test: 21.6430794	best: 21.6430794 (0)	total: 13.8ms	remaining: 41.4s
500:	learn: 20.4119045	test: 20.5617495	best: 20.5617495 (500)	total: 2.52s	remaining: 12.6s
1000:	learn: 19.7925851	test: 20.3289857	best: 20.3289654 (999)	total: 5s	remaining: 9.98s
1500:	learn: 19.4966009	test: 20.2683026	best: 20.2666769 (1483)	total: 7.47s	remaining: 7.46s
bestTest = 20.26667691
bestIteration = 1483
Shrink model to first 1484 iterations.
✅ Fold 1 MAE: 20.2667


Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 22.1267861	test: 20.8013015	best: 20.8013015 (0)	total: 6.56ms	remaining: 19.7s
500:	learn: 20.5497589	test: 19.9968233	best: 19.9968233 (500)	total: 2.59s	remaining: 12.9s
1000:	learn: 19.9741215	test: 19.7978879	best: 19.7978879 (1000)	total: 5.13s	remaining: 10.2s
bestTest = 19.76309931
bestIteration = 1370
Shrink model to first 1371 iterations.
✅ Fold 2 MAE: 19.7631


Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 21.7295655	test: 22.3849894	best: 22.3849894 (0)	total: 6.21ms	remaining: 18.6s
500:	learn: 20.2364588	test: 21.3890817	best: 21.3890817 (500)	total: 2.55s	remaining: 12.7s
1000:	learn: 19.5977594	test: 21.1483416	best: 21.1479492 (997)	total: 5.04s	remaining: 10.1s
1500:	learn: 19.2540224	test: 21.1263718	best: 21.1262265 (1487)	total: 7.57s	remaining: 7.56s
bestTest = 21.12403216
bestIteration = 1572
Shrink model to first 1573 iterations.
✅ Fold 3 MAE: 21.1240


Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 21.9242492	test: 21.6069249	best: 21.6069249 (0)	total: 5.82ms	remaining: 17.5s
500:	learn: 20.5570951	test: 20.2752627	best: 20.2752627 (500)	total: 2.49s	remaining: 12.4s
1000:	learn: 19.9735798	test: 19.8249366	best: 19.8248175 (999)	total: 5.03s	remaining: 10.1s
1500:	learn: 19.6330347	test: 19.6516578	best: 19.6516578 (1500)	total: 7.63s	remaining: 7.62s
2000:	learn: 19.3825395	test: 19.5714518	best: 19.5714518 (2000)	total: 10.2s	remaining: 5.08s
2500:	learn: 19.1345899	test: 19.5484299	best: 19.5438058 (2425)	total: 12.7s	remaining: 2.54s
bestTest = 19.5438058
bestIteration = 2425
Shrink model to first 2426 iterations.
✅ Fold 4 MAE: 19.5438


Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 21.5872938	test: 23.0017264	best: 23.0017264 (0)	total: 6.2ms	remaining: 18.6s
500:	learn: 20.0452893	test: 22.0252918	best: 22.0252918 (500)	total: 2.62s	remaining: 13.1s
1000:	learn: 19.3824698	test: 21.7624425	best: 21.7624425 (1000)	total: 5.24s	remaining: 10.5s
1500:	learn: 18.9921381	test: 21.6640247	best: 21.6640247 (1500)	total: 7.89s	remaining: 7.88s
2000:	learn: 18.7276356	test: 21.6095610	best: 21.6092762 (1999)	total: 10.5s	remaining: 5.25s
2500:	learn: 18.5079708	test: 21.5620960	best: 21.5616833 (2496)	total: 13.1s	remaining: 2.62s
bestTest = 21.55628604
bestIteration = 2663
Shrink model to first 2664 iterations.
✅ Fold 5 MAE: 21.5563

✅ 평균 MAE: 20.4508
📁 제출 파일 저장 완료: submission_catboost_optuna_shap.csv
